# element tracer model for assigning elemental abundances to particles in post-processing

This section is a tutorial on using the element-tracer model to assign elemental abundaces to star particles and gas cells in FIRE-2 and FIRE-3 simulations. This requires that you are analyzing a simulation has has the element-tracer model enabled, via defining GALSF_FB_FIRE_AGE_TRACERS in Gizmo's Config.sh.

In [ ]:
import gizmo_analysis as gizmo
import utilities as ut

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Use this is you are running from within a simulation directory
#simulation_directory = '.'

# use this to point to a specific simulation directory, if you run this notebook from somwhere else
simulation_directory = '/Users/awetzel/work/research/simulation/gizmo/simulations/m12/m12i/elementtracer/m12i_res57000'

In [ ]:
# Read star particles at z = 0 from a simulation with element-tracers enabled

part = gizmo.io.Read.read_snapshots(['star', 'gas'], 'index', 600, assign_hosts_rotation=True, simulation_directory=simulation_directory)

In [ ]:
# The simulation has element-tracers if the following flag is True

part.info['has.elementtracer']

In [ ]:
# Additional information about the element-tracer model used in the simulation

# number of age bins in the element-tracer model
print('number of age bins = {}'.format(part.info['elementtracer.age.bin.number']))

# if element-tracer model used bins equally spaced in log age (which is the default), the min and max age of these bins [Myr] (though GizmoAnalysis will over-ride the age min to be 0)
if 'elementtracer.age.min' in part.info:
    print('age min,max = {}, {} Myr'.format(part.info['elementtracer.age.min'], part.info['elementtracer.age.max']))

# alternately, if the simulation used custom age bins, this lists them
if 'elementtracer.age.bins' in part.info:
    print('age bins = {}'.format(part.info['elementtracer.age.bins']))

# targeted number of element-tracer injection events per age bin
# if <= 0, this means Gizmo deposited weights at each timestep
print('targeted number of events per age bins = {}'.format(part.info['elementtracer.event.number.per.age.bin']))

In [ ]:
# If a simulation has element-tracers enabled, read_snapshots() automatically sets up the nucleosynthetic yields for computing elementa abundances in the FIRE-2 model and assigns the class to the each species' particle dictionary

# Here we explicitly work though these steps for star particles. this works identically for gas cells

#species_name = 'star'
species_name = 'gas'

In [ ]:
# Create and append an element-tracer dictionary class to particle species catalog
# this stores all element-tracer information and provides the methods to compute elemental abundances (mass fractions) for each particle from the element-tracer mass weights

# pass in the snapshot header information, stored via part.info, which contains the element-tracer bin information, to set up the age bins
part[species_name].ElementTracer = gizmo.elementtracer.ElementTracerClass(part.info)

# test of older simulations
#element_index_start = 15
#part[species_name].ElementTracer = gizmo.elementtracer.ElementTracerClass#(element_index_start=element_index_start)
#part[species_name].ElementTracer.assign_age_bins(age_bin_number=16, age_min=1, age_max=14000)
#part[species_name]['massfraction'][:, element_index_start:] *= part.Cosmology['hubble'] / 1e10

In [ ]:
# Set the initial conditions for elemental abundances, that is, the initial mass fraction of each element.
# this is not strictly necessary, the initial massfractions will default to 0 if you do not do this.
# but this step can be useful if you want to be consistent with the metallicity floor (mass fraction = 1e-4 or 1e-5) used in many FIRE-2 simulations.

# you can supply a single float to apply to all abundances (this is the default),
# or you can supply a dictionary with element names as keys and values as initial mass fractions, if you want to use a different initial abundance for each element

metallicity_initial = 1e-5
massfraction_initial = {}
for element_name in FIREYield.NucleosyntheticYield.sun_massfraction:
    massfraction_initial[element_name] = (
       metallicity_initial * FIREYield.NucleosyntheticYield.sun_massfraction[element_name])
part[species_name].ElementTracer.assign_element_massfraction_initial(massfraction_initial)

In [ ]:
# up to now we have not set or assumed any stellar nucleosynthetic rate or yield model, so the information in ElementTracer is completely general

# next we need to assume actual nucleosynthetic rates + yields for a given stellar evolution model, to supply to ElementTracer, to generate actual nucleosynthetic yields from the element-tracer weights.
# see gizmo_elementtracer.py for examples in setting up the required yield class, which should contain a rate + yield model for each element.

# here we use the default stellar evolution model in FIRE-2, assuming a default progenitor metallicity of 1.0 x Solar
FIREYield = gizmo.elementtracer.FIREYieldClass('fire2', progenitor_metallicity=1.0)

In [ ]:
# next we generate a nucleosynthetic yield table, by integrating this nucleosynthetic yield model within each element-tracer age bin, to discretize/average these nucleosynthetic yields within the element-tracer  age bins.
# so, we neet to supply the age bins used in the element-tracer model.
# these yields should be in a dictionary, with element names as keys, and an array of yields within each age bin as values.

yield_dict = FIREYield.get_element_yields(part[species_name].ElementTracer['age.bins'])

In [ ]:
# finally, transfer this yield dictionary to store in ElementTracer.
# this stores the dictionary keys as both the element name and the element symbol, for convenience in calling later.

part[species_name].ElementTracer.assign_element_yields(yield_dict)

In [ ]:
# now you can use .prop() to call the element-tracer elemental mass fractions as derived quantities,
# you just need to append '.elementtracer' to the property name.
# under the hood, this uses ElementTracer.get_element_massfractions() to compute the element mass fractions from the element-tracer weights convolved with the nucleosynthetic yield model, adding in the initial abundances (if you set them).

# mass fraction of iron for each particle, as tracked natively in the simulation
print(part[species_name].prop('massfraction.fe'))

# mass fraction of iron for each particle, as computed via post-processing the element-tracer weights
print(part[species_name].prop('massfraction.elementtracer.fe'))

# 'metallicity' (wrt Solar, as in Asplund et al 2009) of iron for each particle, as tracked natively in the simulation
print(part[species_name].prop('metallicity.fe'))

# 'metallicity' of iron for each particle, as computed via post-processing the element-tracer weights
print(part[species_name].prop('metallicity.elementtracer.fe'))

In [ ]:
#gal = ut.particle.get_galaxy_properties(part, axis_kind='both')

dists = part[species_name].prop('host.distance.principal.cyl')
pis = None
pis = ut.array.get_indices(dists[:, 0], [0, 15], pis)
pis = ut.array.get_indices(dists[:, 2], [-3, 3], pis)

In [ ]:
# plot the results
element_name = 'fe'

from utilities.binning import BinClass

metallicity_limits = [-1.0, 1.0]
Bin = BinClass(metallicity_limits, width=0.01)

metallicity_sim = part[species_name].prop('metallicity.' + element_name, pis)
metallicity_elementtracer = part[species_name].prop('metallicity.elementtracer.' + element_name, pis).clip(-5, 10)

masks = (metallicity_elementtracer > -5) * (metallicity_sim > -5)

distr_sim = Bin.get_distribution(metallicity_sim[masks])
distr_elementtracer = Bin.get_distribution(metallicity_elementtracer[masks])

fig, ax = plt.subplots()
fig.set_size_inches(6, 4)

# plot the native abundances, computed directly in the simulation
ax.plot(distr_sim['bin'], distr_sim['sum'], lw=3, color='black', label='simulation')

# plot the post-processed abundances, from the element-tracer weights
ax.plot(distr_elementtracer['bin'], distr_elementtracer['sum'], lw=3, color='C0', label='element-tracer')

ax.legend(loc='best')
ax.set_ylim(0, None)
ax.set_xlim(metallicity_limits[0], metallicity_limits[1])
ax.set_xlabel('[Fe/H]')

#ut.math.print_statistics(metallicity_sim)

#ut.math.print_statistics(metallicity_elementtracer)

difs = metallicity_elementtracer - metallicity_sim

ut.math.print_statistics(difs)

In [ ]:
# plot the results

from utilities.binning import BinClass

fe_limits = [-1.5, 1.0]
Bin = BinClass(fe_limits, width=0.05)

# plot the native abundances, computed directly in the simulation
fe_sim = part[species_name].prop('metallicity.fe')
alpha_sim = part[species_name].prop('metallicity.o - metallicity.fe')
fe_elementtracer = part[species_name].prop('metallicity.elementtracer.fe').clip(-5, 10)
alpha_elementtracer = part[species_name].prop('metallicity.elementtracer.o').clip(-5, 10) - fe_elementtracer

masks = (fe_sim > -4.5) * (fe_elementtracer > -4.5)

stats_sim = Bin.get_statistics_of_array(fe_sim[masks], alpha_sim[masks])
stats_elementtracer = Bin.get_statistics_of_array(fe_elementtracer[masks], alpha_elementtracer[masks])

fig, ax = plt.subplots()
fig.set_size_inches(6, 4)

ax.plot(stats_sim['bin.mid'], stats_sim['median'], lw=3, color='black', label='simulation')
ax.fill_between(
    stats_sim['bin.mid'], stats_sim['percent.16'], stats_sim['percent.84'], alpha=0.2, lw=3, color='black')

# plot the post-processed abundances, from the element-tracer weights


ax.plot(stats_elementtracer['bin.mid'], stats_elementtracer['median'], lw=3, color='C0', label='element-tracer')
ax.fill_between(
    stats_elementtracer['bin.mid'], stats_elementtracer['percent.16'], stats_elementtracer['percent.84'], alpha=0.2, lw=3, color='C0')
#print(stats['median'])

ax.legend(loc='best')
ax.set_ylim(-0.4, 1.0)
ax.set_xlim(fe_limits[0], fe_limits[1])
ax.set_ylabel('[O/Fe]')
ax.set_xlabel('[Fe/H]')

difs = alpha_elementtracer[masks] - alpha_sim[masks]

ut.math.print_statistics(difs)

## NuGrid

As another example, lets generate the NuGrid tables and re-do the above with those. This
requires having NuPYCee and Sygma installed (https://nugrid.github.io/NuPyCEE/index.html)

In [ ]:
# the NuGrid_yields class accepts kwargs to pass to the underlying
# sygma model, so this allows for the use of the full Symga / NuGrid framework.
#    this simple example just passes metallicity
NuGrid_yield_model = gizmo.elementtracers.NuGrid_yields(
    iniZ = 0.01 # metal mass fraction (must equal a NuGrid table value)
                                                    )


NuGrid_yield_table = gizmo.elementtracers.construct_yield_table(
    NuGrid_yield_model, part.ageprop.age_bins/1000.0)

# again, elements to generate for yield table are arbitary as long as
# they are included in the yield model. Below uses all available elements:
part.set_yield_table(NuGrid_yield_table, [str.lower(x) for x in NuGrid_yield_model.elements])

In [ ]:
# Lets try this out with the NuGrid data now!

from utilities.binning import BinClass

fig,ax=plt.subplots()
fig.set_size_inches(6,4)

bc    = BinClass([-4,2],number= int((2-(-4))/0.1))

# plot the native simulation data:
stats = bc.get_statistics_of_array(
    part['star'].prop('metallicity.fe'),
    part['star'].prop('metallicity.alpha - metallicity.fe')
)


ax.plot(stats['bin.mid'][:-1], stats['median'][:-1], lw=3, color = 'black', label = 'Simulation')
ax.fill_between(stats['bin.mid'], stats['percent.16'], stats['percent.84'], alpha=0.2,
                lw = 3, color='black')


# plot the post-processed data.
# this can be done with just the ".elementtracer" string, which works
# for all things that the elements work on already (metallicity, mass, massfraction, etc.)
bc    = BinClass([-4,2],number= int((2-(-4))/0.1))
stats = bc.get_statistics_of_array(part['star'].prop('metallicity.elementtracer.fe'),
                                   part['star'].prop('metallicity.elementtracer.alpha - metallicity.elementtracer.fe'))


ax.plot(stats['bin.mid'][:-1], stats['median'][:-1], lw=3, color = 'C0', label = 'NuGrid - solar')
ax.fill_between(stats['bin.mid'], stats['percent.16'], stats['percent.84'], alpha=0.2,
                lw = 3, color='C0')


ax.legend(loc='best')
ax.set_ylim(-1,1)
ax.set_xlim(-4,2)
ax.set_ylabel(r'[$\alpha$/Fe]')
ax.set_xlabel('[Fe/H]')